<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Classification using Vantage and Google Gemini
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Revolutionize customer complaint resolution with our pioneering solution, which seamlessly integrates the capabilities of <b>Teradata Vantage</b> and <b>Google Gemini</b> model as LLM. This powerful synergy enables businesses to classify customer complaints with unmatched precision and speed, allowing them to swiftly identify and address concerns, thereby elevating overall customer satisfaction and loyalty.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Key Features:</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Automated Classification:</b> Our AI-driven model categorizes complaints into predefined categories, ensuring consistency and reducing manual effort.</li>
    <li><b>Contextual Understanding:</b> The system comprehends the nuances of customer feedback, capturing subtle differences in tone and language.</li>
    <li><b>Real-time Insights:</b> Generate instant reports and analytics to identify trends, patterns, and areas for improvement.</li>
</ul>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Benefits:</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Enhanced Customer Experience:</b> Swiftly address customer concerns, fostering trust and loyalty.</li>
    <li><b>Improved Operational Efficiency:</b> Reduce manual processing time, allowing teams to focus on high-value tasks.</li>
    <li><b>Data-Driven Decision Making: </b> Make informed decisions with actionable insights from complaint data.</li>
</ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'>Experience the transformative power of Generative AI in complaints classification.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Setup API key for Google Gemini</li>
    <li>Classify Compalints</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>1. Configuring the environment</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Install the required libraries</b></p>

In [ ]:
%%capture

!pip install -r requirements.txt --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import numpy as np
import pandas as pd
import timeit
import boto3
from tqdm import tqdm
from teradataml import *
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.subplots as subplots

# GenAI libs
import google.generativeai as genai
from google.generativeai import protos


display.max_rows = 5
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 5)
from IPython.display import display, Markdown

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Connect to Vantage</b>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>2.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO=Complaints_Classification_Gemini.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>2.2 Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_local');"        # Takes 2 minutes

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Setup API key for Google Gemini</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Please enter the Google API Key, if you don't have one, please get it from <a href = 'https://ai.google.dev/gemini-api/docs/api-key'>here</a></p>

In [ ]:
GOOGLE_API_KEY = getpass.getpass(prompt = 'Please enter GOOGLE_API_KEY: ')
genai.configure(api_key = GOOGLE_API_KEY)

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>3.1. Define the Gemini model and Prompt</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following section defines the type of Gemini model used. Here we use <b>gemini-1.5-pro-latest</b></p>

In [ ]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

model = genai.GenerativeModel(
    model_name = "models/gemini-1.5-pro-latest"
)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Classify Complaints</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We'll use a sample of the data to classify complaints</p>

In [ ]:
tdf = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Consumer_Complaints'))
tdf = tdf.assign(id = tdf.complaint_id).drop('complaint_id', axis = 1)
tdf

In [ ]:
# take sample of records to go quickly
df = tdf.sample(50).to_pandas()
df['Prediction'] = ""
df['Reasoning with Chain of Thought'] = ""

for i in tqdm(range(len(df))):
    try:
        prompt = f'''
        User prompt:
        The following is text from a review:

        “{df['consumer_complaint_narrative'][i]}”

        Give me reasoning as well as Category for this review

        Instructions for Reasoning:
        - Give me Reasoning in detail
        - Only one sentence reasoning
        Instructions for Category:
        - The review falls into one of the following categories: Complaint, Non-Complaint
        - Select one category from the given ones
        - Important: Do not add any formating into the output. For example ** Complaint or **Complaint** refrain from such format

        Final output comes in the format:
        Category: 
        Reasoning: 
        '''
        output = model.generate_content([prompt])
        finish_reason = output.candidates[0].finish_reason

        if finish_reason ==  protos.Candidate.FinishReason.STOP:
            output = output.candidates[0].content.parts[0].text
            # apply regex
            category = re.search('Category:(.*)', output).group(1)
            reasoning = re.search('Reasoning:(.*)', output).group(1)
        else:
            category = "Non-Complaint"
            reasoning = ""

        df['Prediction'][i] = category
        df['Reasoning with Chain of Thought'][i] = reasoning
    except:
        pass

In [ ]:
df['Prediction'] = df['Prediction'].apply(lambda x: x.strip())
df['Reasoning with Chain of Thought'] = df['Reasoning with Chain of Thought'].apply(lambda x: x.strip())

In [ ]:
df[['id', 'consumer_complaint_narrative', 'Prediction', 'Reasoning with Chain of Thought']].head(5)

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.1 Consumer Complaints Prediction vs Occurrences</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'>A graph illustrating the relationship between consumer complaints prediction and the number of occurrences. This visual representation helps identify trends, patterns, and areas for improvement, enabling data-driven decision making.</p>

In [ ]:
from collections import Counter
data = Counter(df['Prediction'])

# Convert Counter data to DataFrame
viz_df = pd.DataFrame.from_dict(data, orient='index', columns=['Count']).reset_index()

# Rename columns
viz_df.columns = ['Prediction', 'Count']

# Create bar graph using Plotly Express
fig = px.bar(viz_df, x='Prediction', y='Count', color='Prediction',
             labels={'Count': 'Number of Occurrences', 'Prediction': 'Prediction'})

# Show the plot
fig.show()

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.2 Word Cloud for Consumer Complaints Prediction</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'>A visual representation of <b>consumer complaints prediction</b>, highlighting the most frequent words and pain points in customer feedback. This word cloud helps identify trends, sentiment, and areas for improvement, enabling data-driven decision making.</p>

In [ ]:
def display_helper(msg):
    return display(Markdown(
        f"""<div class="alert alert-block alert-info">
        <p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b>
        <i>{msg}</i></p>"""))

In [ ]:
complaint = df[df['Prediction'] == 'Complaint']
complaint_text = ' '.join(complaint['consumer_complaint_narrative'])

# Replace 'X' with blank space
modified_string = complaint_text.replace('X', '')

if len(modified_string):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(modified_string)

    # Display the word cloud
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title("Complaints")
    plt.tight_layout()
    plt.axis("off")
    plt.show()
else:
    display_helper("""We included complaint and non-complaint options for completeness. 
    It's possible that our sample dataset doesn't contain any actual complaints.""")

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.3 Word Cloud for Non-Complaints Prediction</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'>A visual representation of <b>non-complaints prediction</b>, highlighting the most frequent words and positive sentiments in customer feedback. This word cloud helps identify trends, sentiment, and areas of satisfaction, enabling data-driven decision making.</p>

In [ ]:
non_complaint = df[df['Prediction'] == 'Non-Complaint']
non_complaint_text = ' '.join(non_complaint['consumer_complaint_narrative'])

# Replace 'X' with blank space
modified_string = non_complaint_text.replace('X', '')

if len(modified_string):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(modified_string)

    # Display the word cloud
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title("Non-Complaints")
    plt.tight_layout()
    plt.axis("off")
    plt.show()
else:
    display_helper("""We included both complaint and non-complaint options for completeness. 
          But since this is a complaints dataset, we don't expect to see any <b>non-complaints</b>.""")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now the results can be saved back to Vantage.</p>

In [ ]:
copy_to_sql(df = df, table_name = 'reviews', if_exists = 'replace')

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ComplaintAnalysis');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial; color: #00233C;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>